# Linear Models

# Explain the '<' pattern (article+data)
# Build geoapp where you can increase a radius centered at IU (Iowa State University), then see the predicted house prices + actual house prices. Once click on the house, can see the difference explained by examining that house's shap value! 
# Use CatBoost's Shap value to build a Dashboard

# Blend predictions??


In [1]:
KAGGLE=False # set to True if using the Kaggle version of the dataset

In [62]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import sklearn.preprocessing

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn import preprocessing
#from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold,KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, HuberRegressor

from sklearn.metrics import r2_score

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [3]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 400)

The data has been pre-processed

In [4]:
def load_data(tree=None):
    if KAGGLE:
        train = pd.read_csv('data/train.csv')
        test = pd.read_csv('data/test.csv')
        num_train=np.shape(train)[0]
        num_test=np.shape(test)[0]
        alldata=train.append(test, ignore_index=True)
    else:
        alldata=pd.read_csv('./data/Ames_HousePriceLocsDistNoDpl.csv')#, index_col=0) #already cleaned (see work in appendix files)
    return alldata

alldata=load_data()

In [5]:
alldata.head()

,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,lats,longs,MissingLoc,distToUI
0,856,126000,30,RL,65.556308,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal,42.018564,-93.651619,0,0.620685
1,1049,139500,120,RL,42.000000,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,42.024855,-93.663671,0,0.890758
2,1001,124900,30,C (all),60.000000,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal,42.021548,-93.614068,0,1.702803
3,1039,114000,70,RL,80.000000,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal,42.037391,-93.612207,0,1.909280
4,1665,227000,60,RL,70.000000,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal,42.044554,-93.631818,0,1.445916


In [6]:
nums = [col for col in alldata.columns if alldata.dtypes[col] != 'object']
cats = [col for col in alldata.columns if alldata.dtypes[col] == 'object']
alldata=pd.get_dummies(alldata).reset_index(drop=True)

In [7]:
alldata['LogPrice']=np.log1p(alldata.SalePrice)

In [8]:
alldata.distToUI.head()

0    0.620685
1    0.890758
2    1.702803
3    1.909280
4    1.445916
Name: distToUI, dtype: float64

In [9]:
def split_into_train_test(alldata):
    y=alldata['LogPrice']
    X=alldata.drop(['SalePrice','LogPrice'],axis=1)
    from sklearn.model_selection import train_test_split
    if KAGGLE:
        return split_train_test(alldata=alldata,num_train=num_train)
    else:
        return train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = split_into_train_test(alldata)  

The sale price appears to be skewed to the right

In [10]:
# from sklearn.pipeline import make_pipeline 
# from sklearn.preprocessing import PolynomialFeatures, StandardScaler
# pipe = make_pipeline(OneHotEncoder(),StandardScaler(),PolynomialFeatures(3),LinearRegression())
# run_model(pipe,X_train,y_train,X_test,y_test)

In [11]:
def run_model(model,X_train,y_train,X_test,y_test):
    
    # Fit Model
    model.fit(X_train,y_train)
    
    # Get Metrics
    
    preds = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test,preds))
    print(f'RMSE on Test: {rmse}')
    r2 = r2_score(y_test, preds)
    print(f'R^2 on Test: {r2}')

    return model

### Without log price

In [12]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
run_model(model_lr,X_train,y_train,X_test,y_test)

RMSE on Test: 1568.3340736844364
R^2 on Test: -15812600.201752799


LinearRegression()

### With log price

In [13]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
run_model(model_lr,X_train,y_train,X_test,y_test)

RMSE on Test: 1568.3340736844364
R^2 on Test: -15812600.201752799


LinearRegression()

#### Size-related columns

In [14]:
size_related = alldata.filter(regex='SF$|Area$')


In [15]:
size_related.head()

,GrLivArea,LotArea,MasVnrArea,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea
0,856,7890,0.0,618.0,856.0,856,0,0,399.0,0,0,0
1,1049,4235,149.0,104.0,1049.0,1049,0,0,266.0,0,105,0
2,1001,6060,0.0,100.0,837.0,1001,0,0,216.0,154,0,0
3,1039,8146,0.0,405.0,405.0,717,322,0,281.0,0,0,0
4,1665,8400,0.0,167.0,810.0,810,855,0,528.0,0,45,0


In [16]:
size_cols=set(size_related.columns)

#### Neighborhood columns

In [17]:
ngh_related = alldata.filter(regex='^Neighborhood')


In [18]:
ngh_related.head()

,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [19]:
ngh_cols=set(ngh_related.columns)

#### Condition columns

In [20]:
cond_related = alldata.filter(regex='^Condition|^ExterCond|^BsmtCond')


In [21]:
cond_related.head()

,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,BsmtCond_Ex,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA
0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [22]:
cond_cols=set(cond_related.columns)

In [23]:
qual_related = alldata.filter(regex='^KitchenQual')  #|^BasementQual')


In [24]:
qual_related.head()

,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po,KitchenQual_TA
0,0,0,0,0,1
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,0,0,1
4,0,0,1,0,0


In [25]:
qual_cols=set(qual_related.columns)

In [26]:
street_related = alldata.filter(regex='^Street')


In [27]:
street_related.head()

,Street_Grvl,Street_Pave
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [28]:
street_cols=set(street_related.columns)

#### Garage-related columns

In [29]:
garage_related = alldata.filter(regex='^Garage') #extra cols will be dropped when we make the set


In [30]:
garage_related.head()

,GarageYrBlt,GarageCars,GarageArea,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA
0,1939.0,2.0,399.0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1
1,1984.0,1.0,266.0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
2,1930.0,1.0,216.0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0
3,1940.0,1.0,281.0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1
4,2001.0,2.0,528.0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1


In [31]:
garage_cols=set(garage_related.columns)

In [32]:
air_related = alldata.filter(regex='^Central') #extra cols will be dropped when we make the set


In [33]:
air_related.head()

,CentralAir_N,CentralAir_Y
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [34]:
air_cols=set(air_related.columns)

In [35]:
fireplace_related = alldata.filter(regex='^Fireplace') #extra cols will be dropped when we make the set


In [36]:
fireplace_related.head()

,Fireplaces,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA
0,1,0,0,1,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [37]:
fireplace_cols=set(fireplace_related.columns)

In [38]:
salecondition_related = alldata.filter(regex='^SaleCondition') #extra cols will be dropped when we make the set


In [39]:
salecondition_related.head()

,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0


In [40]:
salecondition_cols=set(salecondition_related.columns)

In [41]:
ssnporch_related = alldata.filter(regex='^3SsnPorch') #extra cols will be dropped when we make the set


In [42]:
ssnporch_related.head()

,3SsnPorch
0,0
1,0
2,86
3,0
4,0


In [43]:
ssnporch_cols=set(ssnporch_related.columns)

In [44]:
zoning_related = alldata.filter(regex='^MSZoning') #extra cols will be dropped when we make the set

In [45]:
zoning_related.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,1,0,0,0,0,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0


In [46]:
zoning_cols=set(zoning_related.columns)

In [47]:
bldg_related = alldata.filter(regex='^BuildingType') #extra cols will be dropped when we make the set

In [48]:
bldg_related.head()

""
0
1
2
3
4


In [49]:
bldg_cols=set(bldg_related.columns)

In [50]:
feature_imp_cols=set(['OverallQual','OverallCond','lats','longs','YearBuilt','YearRemodAdd','GarageCars','Fireplaces','LotArea','LotFrontage'])#,'BsmtFullBath'])

In [51]:
alldata.head()

,GrLivArea,SalePrice,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,lats,longs,MissingLoc,distToUI,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSewr,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_Po,BsmtQual_TA,BsmtCond_Ex,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageF

In [52]:
alldata.columns

Index(['GrLivArea', 'SalePrice', 'MSSubClass', 'LotFrontage', 'LotArea',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       ...
       'SaleType_Oth', 'SaleType_VWD', 'SaleType_WD ', 'SaleCondition_Abnorml',
       'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family',
       'SaleCondition_Normal', 'SaleCondition_Partial', 'LogPrice'],
      dtype='object', length=299)

In [53]:
price_col=set(['LogPrice','SalePrice']) #will be dropped in train-valid split

In [54]:
final_cols=size_cols|feature_imp_cols|ngh_cols|cond_cols|qual_cols|price_col|street_cols|garage_cols|air_cols#|bldg_cols#|basement_kitchen_cols#|zoning_cols#|ssnporch_cols#|fireplace_cols|salecondition_cols

In [55]:
#final_cols=size_cols|ngh_cols|price_col

In [56]:
final_cols

{'1stFlrSF',
 '2ndFlrSF',
 'BsmtCond_Ex',
 'BsmtCond_Fa',
 'BsmtCond_Gd',
 'BsmtCond_Po',
 'BsmtCond_TA',
 'BsmtUnfSF',
 'CentralAir_N',
 'CentralAir_Y',
 'Condition1_Artery',
 'Condition1_Feedr',
 'Condition1_Norm',
 'Condition1_PosA',
 'Condition1_PosN',
 'Condition1_RRAe',
 'Condition1_RRAn',
 'Condition1_RRNe',
 'Condition1_RRNn',
 'Condition2_Artery',
 'Condition2_Feedr',
 'Condition2_Norm',
 'Condition2_PosA',
 'Condition2_PosN',
 'Condition2_RRAe',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'ExterCond_Ex',
 'ExterCond_Fa',
 'ExterCond_Gd',
 'ExterCond_Po',
 'ExterCond_TA',
 'Fireplaces',
 'GarageArea',
 'GarageCars',
 'GarageCond_Ex',
 'GarageCond_Fa',
 'GarageCond_Gd',
 'GarageCond_Po',
 'GarageCond_TA',
 'GarageFinish_Fin',
 'GarageFinish_RFn',
 'GarageFinish_Unf',
 'GarageQual_Ex',
 'GarageQual_Fa',
 'GarageQual_Gd',
 'GarageQual_Po',
 'GarageQual_TA',
 'GarageType_2Types',
 'GarageType_Attchd',
 'GarageType_Basment',
 'GarageType_BuiltIn',
 'GarageType_CarPort',
 'GarageType_D

In [57]:
### Pipeline

In [58]:
X_train, X_test, y_train, y_test = split_into_train_test(alldata[list(final_cols)])  

In [59]:
model_lr = LinearRegression()
run_model(model_lr,X_train,y_train,X_test,y_test)

RMSE on Test: 0.12168755487761167
R^2 on Test: 0.9048040458051534


LinearRegression()

In [59]:
model_lr = HuberRegressor()
run_model(model_lr,X_train,y_train,X_test,y_test)

RMSE on Test: 0.12168755487761167
R^2 on Test: 0.9048040458051534


LinearRegression()

In [60]:
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RepeatedKFold
import warnings
warnings.filterwarnings('ignore')


cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
# define model
ratios = np.arange(0, 1, 0.01)
alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
model = ElasticNetCV(l1_ratio=ratios, alphas=alphas, cv=cv, n_jobs=-1)
# fit model
model.fit(X_train, y_train)


ElasticNetCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1, 0.0, 1.0, 10.0, 100.0],
             cv=RepeatedKFold(n_repeats=3, n_splits=5, random_state=42),
             l1_ratio=array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99]),
             n_jobs=-1)

In [61]:
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test,preds))
print(f'RMSE on Test: {rmse}')
r2 = r2_score(y_test, preds)
print(f'R^2 on Test: {r2}')

RMSE on Test: 0.12395349566594446
R^2 on Test: 0.9012257547314438


In [306]:
model.coef_

array([-3.04633326e-02, -0.00000000e+00,  5.88201384e-02,  1.18622430e-02,
        5.73964420e-02,  5.50220263e-04, -1.78325648e-03, -6.09417136e-03,
        5.93755208e-02,  1.85028910e-02, -2.81846398e-02,  0.00000000e+00,
       -5.98653239e-03, -4.33826728e-02,  6.91404455e-02,  2.89641388e-02,
       -6.62557860e-06,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  8.68894499e-03, -0.00000000e+00, -1.85077448e-02,
        2.44945933e-02, -0.00000000e+00, -9.35775069e-03, -3.97803482e-02,
       -0.00000000e+00,  7.85616797e-05,  2.47225073e-02,  2.06517937e-03,
        4.82843016e-02,  6.63264848e-02,  2.56720294e-04,  0.00000000e+00,
       -8.70149724e-05,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -1.37076984e-02, -4.51765012e-02,  0.00000000e+00,  9.46062298e-03,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -2.28651000e-02,
        2.04199534e-06, -0.00000000e+00,  2.02627425e-04,  0.00000000e+00,
       -1.20990775e-01,  

In [308]:
import pickle
pickle.dump(model, open('./models/model_elastic', 'wb'))


## As an experiment, try fitting on all data with statsmodels (since that approach is more of stats rather than ml)

In [ ]:
X=alldata[list(final_cols)]
y=alldata['LogPrice']
X=alldata.drop(['SalePrice','LogPrice'],axis=1)

In [295]:
import statsmodels.api as sm
x = sm.add_constant(X)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(y,x)
results = model.fit()
print(results.summary()) # How to print out the summary report

                            OLS Regression Results                            
Dep. Variable:               LogPrice   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.935
Method:                 Least Squares   F-statistic:                     143.7
Date:                Sat, 13 Aug 2022   Prob (F-statistic):               0.00
Time:                        09:35:20   Log-Likelihood:                 2479.8
No. Observations:                2603   AIC:                            -4434.
Df Residuals:                    2340   BIC:                            -2891.
Df Model:                         262                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.57

In [298]:
import statsmodels.api as sm
x = sm.add_constant(X)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(y,x)
results = model.fit_regularized(method='elastic_net', alpha=0.0, L1_wt=1.0, start_params=None, profile_scale=False, refit=False)

print(results.summary()) # How to print out the summary report

NotImplementedError: 

In [293]:
import statsmodels.api as sm
x = sm.add_constant(X_train)
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(y_train,x)
results = model.fit()
print(results.summary()) # How to print out the summary report

                            OLS Regression Results                            
Dep. Variable:               LogPrice   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     238.4
Date:                Sat, 13 Aug 2022   Prob (F-statistic):               0.00
Time:                        09:31:20   Log-Likelihood:                 1620.6
No. Observations:                2082   AIC:                            -3057.
Df Residuals:                    1990   BIC:                            -2538.
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    1.9735 

In [100]:
# pipe = make_pipeline(StandardScaler(),PolynomialFeatures(2),LinearRegression())
# model_poly=run_model(pipe,X_train,y_train,X_test,y_test)